In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from data_read import load_data_into_dataframe

#enable IPython to display matplolib graphs
%matplotlib inline

In [15]:
h_df = load_data_into_dataframe("halfhourly")
h_df.head()

/home/daniel/Bachelor/Bachelor_thesis/data_read.py:39: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=True)
/home/daniel/Bachelor/Bachelor_thesis/data_read.py:39: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=True)
/home/daniel/Bachelor/Bachelor_thesis/data_read.py:39: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=True)
/home/daniel/Bachelor/Bachelor_thesis/data_read.py:39: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=True)
/home/daniel/Bachelor/Bachelor_t

,LCLid,tstp,energy(kWh/hh)
0,MAC000239,2011-12-02 10:00:00.0000000,0.261
1,MAC000239,2011-12-02 10:30:00.0000000,0.138
2,MAC000239,2011-12-02 11:00:00.0000000,0.089
3,MAC000239,2011-12-02 11:30:00.0000000,0.1
4,MAC000239,2011-12-02 12:00:00.0000000,0.236


In [16]:
h_df.shape

(167817021, 3)

In [18]:
h_df["tstp"]

0          2011-12-02 10:00:00.0000000
1          2011-12-02 10:30:00.0000000
2          2011-12-02 11:00:00.0000000
3          2011-12-02 11:30:00.0000000
4          2011-12-02 12:00:00.0000000
                      ...             
1528553    2013-09-09 22:00:00.0000000
1528554    2013-09-09 22:30:00.0000000
1528555    2013-09-09 23:00:00.0000000
1528556    2013-09-09 23:30:00.0000000
1528557    2013-09-10 00:00:00.0000000
Name: tstp, Length: 167817021, dtype: object

In [19]:
h_df.loc[0,"tstp"]

ERROR! Session/line number was not unique in database. History logging moved to new session 139


0    2011-12-02 10:00:00.0000000
0    2011-12-14 14:00:00.0000000
0    2011-11-23 12:30:00.0000000
0    2011-12-12 11:00:00.0000000
0    2011-11-23 13:00:00.0000000
                ...             
0    2012-04-27 12:00:00.0000000
0    2011-12-07 10:00:00.0000000
0    2011-12-06 09:30:00.0000000
0    2012-03-02 11:30:00.0000000
0    2011-12-07 11:30:00.0000000
Name: tstp, Length: 112, dtype: object